In [16]:
from __future__                           import print_function
%matplotlib inline

import re
import os
import sys
import h5py

import matplotlib.pyplot                  as plt
import numpy                              as np
import random                             as rd
import tables                             as tb
import tensorflow                         as tf

from glob                                 import glob
from matplotlib.patches                   import Ellipse


# Keras imports
import keras.backend.tensorflow_backend   as   K

from keras                                import regularizers
from keras                                import callbacks

from keras.models                         import Model, load_model

from keras.layers                         import Input, Dense, MaxPooling3D, AveragePooling3D
from keras.layers                         import Conv3D, Conv2D, AveragePooling2D, Activation, Dropout, ZeroPadding3D
from keras.layers                         import Add
from keras.layers                         import Flatten, BatchNormalization

from keras.layers.normalization           import BatchNormalization
from keras.layers.convolutional           import Convolution2D, MaxPooling2D
from keras.layers.core                    import Flatten

from keras.optimizers                     import SGD, Adam, Nadam, Adadelta 

from keras.regularizers                   import l2, l1
from keras.initializers                   import RandomNormal

from keras.utils.layer_utils              import print_summary


#os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'

### Create a combined file from several .npz files

In [ ]:
npz_dir = "/home/amenkara/petalo_files_26_nov/"

# input/output files

files = glob(npz_dir + '*.npz')

files = sorted(files)
print(files)

out_file = "combined_file_26nov.npz"

X_    = []
Compt_= []
Var_  = []
Mean_ = [] 
Ev_   = []


for f in files:

    fn = np.load(f)
    
    if(len(X_) == 0):
        X_     = fn['X_train']
        Compt_ =  fn['Compt_train']
        Var_   =  fn['Varpos_train']
        Mean_  =  fn['Meanpos_train']
        Ev_    =  fn['Ev_train']
    else:
        X_     = np.concatenate((X_,     fn['X_train'])      )
        Compt_ = np.concatenate((Compt_, fn['Compt_train'])  ) 
        Var_   = np.concatenate((Var_,   fn['Varpos_train']) )
        Mean_  = np.concatenate((Mean_,  fn['Meanpos_train']))
        Ev_    = np.concatenate((Ev_,    fn['Ev_train'])     )
    
print("Saving combined file")

np.savez(out_file,
        X_=X_, Compt_= Compt_, Var_=Var_, Mean_=Mean_, Ev_=Ev_)

### Load the data from the dataset

In [2]:
out_file = "training_set_test_pitch4mm_file0.npz"

In [3]:
data = np.load(out_file)
X_, Compt_, Ev_ = data['images'],data['labels'], data['event_number']

In [4]:
len(X_)

1011

### Separate train, validation and test set

In [5]:
train_cut = int(0.8*len(X_))
val_cut   = int(0.9*len(X_))

X_train      = X_[:train_cut]
X_val        = X_[train_cut:val_cut] 
X_test       = X_[val_cut:]

Compt_train  =  Compt_[:train_cut]
Compt_val    =  Compt_[train_cut:val_cut]
Compt_test   =  Compt_[val_cut:]

### Define the model

In [6]:
K.clear_session()

In [7]:
def model_CNN(inputs):
    #el 32 implica que hay 32 filtros de 5x5
    cinputs = Conv2D(96, (5, 5), padding='same', strides=(2, 2), activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(0.01),  activity_regularizer=regularizers.l1(0.01))(inputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(0.01),  activity_regularizer=regularizers.l1(0.01))(cinputs)
    cinputs = Dropout(.2)(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    cinputs = Conv2D(384, (2, 2), padding='same', strides=(1, 1), activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(0.01), activity_regularizer=regularizers.l1(0.01))(cinputs)
    cinputs = Dropout(.2)(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    
    f1 = Flatten()(cinputs)
    f1 = Dense(units=64, activation='relu', kernel_initializer='he_normal')(f1)
    #f1 = Dropout(.2)(f1)

    inc_output = Dense(units=1, activation='sigmoid', kernel_initializer='lecun_normal')(f1)
    
    model = Model(inputs, inc_output)
    
    model.compile(loss='binary_crossentropy',
                      optimizer=Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999,
                                      epsilon=1e-08, schedule_decay=0.001), metrics=['accuracy'])
    
    return model

In [8]:
dim     = 30

input_  = Input((dim,dim,1))
model   = model_CNN(input_)

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30, 30, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 96)        2496      
_________________________________________________________________
average_pooling2d_1 (Average (None, 8, 8, 96)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 96)          384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 192)         166080    
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 192)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 192)         768       
__________

In [10]:
tboard = callbacks.TensorBoard(log_dir='./logs/test_file0_v1', histogram_freq=0, write_graph=True, write_images=False)

### Training

In [11]:
model.fit(X_train, Compt_train, batch_size=18, epochs=20, shuffle=False, callbacks=[tboard],verbose=1, validation_data=(X_val, Compt_val))

Train on 808 samples, validate on 101 samples
Epoch 1/20
808/808 [==============================] - 3s 3ms/step - loss: 735.5822 - acc: 0.6188 - val_loss: 538.8776 - val_acc: 0.6832
Epoch 2/20
808/808 [==============================] - 2s 2ms/step - loss: 531.1044 - acc: 0.6696 - val_loss: 427.5538 - val_acc: 0.6733
Epoch 3/20
808/808 [==============================] - 2s 2ms/step - loss: 450.2987 - acc: 0.6906 - val_loss: 370.8260 - val_acc: 0.6733
Epoch 4/20
808/808 [==============================] - 2s 2ms/step - loss: 393.2664 - acc: 0.7005 - val_loss: 328.4942 - val_acc: 0.6733
Epoch 5/20
808/808 [==============================] - 2s 2ms/step - loss: 344.3921 - acc: 0.6993 - val_loss: 293.4603 - val_acc: 0.6832
Epoch 6/20
808/808 [==============================] - 2s 2ms/step - loss: 301.4207 - acc: 0.7030 - val_loss: 264.1526 - val_acc: 0.6832
Epoch 7/20
808/808 [==============================] - 2s 2ms/step - loss: 267.4056 - acc: 0.7054 - val_loss: 240.8612 - val_acc: 0.6733
Ep

In [12]:
prediction = model.predict(X_test, batch_size=18, verbose=0)

In [13]:
print(prediction)

[[0.678588  ]
 [0.678588  ]
 [0.6801706 ]
 [0.678588  ]
 [0.678588  ]
 [0.6707339 ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.689354  ]
 [0.678588  ]
 [0.5646102 ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.6795197 ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.5942508 ]
 [0.678588  ]
 [0.67789775]
 [0.678588  ]
 [0.67336655]
 [0.678588  ]
 [0.678588  ]
 [0.6649633 ]
 [0.6781243 ]
 [0.6682466 ]
 [0.678588  ]
 [0.678588  ]
 [0.6644964 ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.6793885 ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.7028598 ]
 [0.678588  ]
 [0.678588  ]
 [0.6786005 ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.678588  ]
 [0.677392  ]
 [0.678588  ]
 [0.678588  ]
 [0.9205143 ]
 [0.67

In [ ]:
def plot_result(cut):
    elements = []
    for element in prediction:
        if element > cut:
            elements.append(0)
        else:
            elements.append(1)
        
    aciertos = 0
    total = len(elements)
    for i in range(total):
        if elements[i]== Compt_test[i]:
            aciertos += 1
        
    print(aciertos)    
    return aciertos/total

In [ ]:
matrix = []
for i in range(500, 1000):
    matrix.append([i/1000., plot_result(i/1000.)])
        

In [ ]:
np.transpose(matrix)[1]

In [ ]:
plt.plot(np.transpose(matrix)[0][:], np.transpose(matrix)[1][:])

In [ ]:
score = []


In [18]:
model.evaluate(x=X_test, y=Compt_test, batch_size=None, verbose=1, sample_weight=None, steps=None)

102/102 [==============================] - 0s 2ms/step


[156.75977310479857, 0.6372549025451436]